<a href="https://colab.research.google.com/github/vnavya2004/Depressiondetection_BTP/blob/main/mix(spanishroberta%2Cmultilingualbet).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Roberta spanish
!pip install tensorflow_text
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Upload the file
uploaded = files.upload()
df = pd.read_csv("spanish.csv", encoding='latin1')  # or specify the correct encoding of your CSV file

X_train, X_test, y_train, y_test = train_test_split(df['traducido'], df['class'], stratify=df['class'])

X_train = X_train.astype(str)
X_test = X_test.astype(str)

y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

# Load BERT pre-trained models
bert_preprocess = hub.KerasLayer("https://kaggle.com/models/kaggle/xlm-roberta/frameworks/TensorFlow2/variations/multi-cased-preprocess/versions/1")
bert_encoder = hub.KerasLayer("https://www.kaggle.com/models/kaggle/xlm-roberta/frameworks/TensorFlow2/variations/multi-cased-l-12-h-768-a-12/versions/1")

# Build the model
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Additional LSTM layer
reshaped_output = tf.keras.layers.Reshape((1, 768))(outputs['pooled_output'])
lstm_layer = tf.keras.layers.LSTM(128, name='lstm')(reshaped_output)

# Dropout layer
dropout_layer = tf.keras.layers.Dropout(0.1, name="dropout")(lstm_layer)

# Output layer
output_layer = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(dropout_layer)

# Create the model
model = tf.keras.Model(inputs=[text_input], outputs=output_layer)
model.summary()

# Compile and train the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Consider increasing the batch size
model.fit(X_train, y_train, epochs=5, batch_size=32)

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 20.4 MB/s eta 0:00:00


Saving spanish.csv to spanish.csv
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_word_ids': (None,    0         ['text[0][0]']                
                             128),                                                                
                              'input_type_ids': (None,                                            
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             }                              

In [ ]:
# run this way for all 5
#multilingualBert Arabic
!pip install tensorflow_text
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

uploaded = files.upload()
df = pd.read_excel(pd.ExcelFile(list(uploaded.keys())[0]), header=0)

X_train, X_test, y_train, y_test = train_test_split(df['tweet'], df['label'], stratify=df['label'])

X_train = X_train.astype(str)
X_test = X_test.astype(str)

y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

# Load BERT pre-trained models
bert_preprocess = hub.KerasLayer("https://www.kaggle.com/models/keras/bert/frameworks/Keras/variations/bert_base_multi")
bert_encoder = hub.KerasLayer("https://www.kaggle.com/models/keras/bert/frameworks/Keras/variations/bert_base_multi")

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

model = tf.keras.Model(inputs=[text_input], outputs=[l])
model.summary()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5)

# Predict on the testing data
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# Convert y_test to int for comparison
y_test = y_test.astype(int)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


In [ ]:

#multilingualBert Bangla 2
!pip install tensorflow_text
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils import class_weight

uploaded = files.upload()
df = pd.read_excel(pd.ExcelFile(list(uploaded.keys())[0]), header=0)

# Display class distribution
print("Class Distribution:")
print(df['labels'].value_counts())

X_train, X_test, y_train, y_test = train_test_split(df['tweets'], df['labels'], stratify=df['labels'])

X_train = X_train.astype(str)
X_test = X_test.astype(str)

y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

# Calculate class weights to handle imbalanced data
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

bert_preprocess = hub.KerasLayer("https://www.kaggle.com/models/keras/bert/frameworks/Keras/variations/bert_base_multi")
bert_encoder = hub.KerasLayer("https://www.kaggle.com/models/keras/bert/frameworks/Keras/variations/bert_base_multi")

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

model = tf.keras.Model(inputs=[text_input], outputs=[l])
model.summary()

Metrics = [tf.keras.metrics.BinaryAccuracy(name='accuracy'),
           tf.keras.metrics.Precision(name='precision'),
           tf.keras.metrics.Recall(name='recall')]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model with class weights
model.fit(X_train, y_train, epochs=5, class_weight=class_weight_dict)

# Predict on the testing data
y_pred = model.predict(X_test)
# Experiment with different threshold values
threshold = 0.5
y_pred = (y_pred > threshold).astype(int)

# Convert y_test to int for comparison
y_test = y_test.astype(int)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


In [ ]:
#MultilingualBert Japaneese
!pip install tensorflow_text
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

uploaded = files.upload()
df = pd.read_excel(pd.ExcelFile(list(uploaded.keys())[0]), header=0)

df.head(5)

X_train, X_test, y_train, y_test = train_test_split(df['tweets'], df['labels'], stratify=df['labels'])

X_train = X_train.astype(str)
X_test = X_test.astype(str)

y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

bert_preprocess = hub.KerasLayer("https://www.kaggle.com/models/keras/bert/frameworks/Keras/variations/bert_base_multi")
bert_encoder = hub.KerasLayer("https://www.kaggle.com/models/keras/bert/frameworks/Keras/variations/bert_base_multi")

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

model = tf.keras.Model(inputs=[text_input], outputs=[l])
model.summary()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5)

# Predict on the testing data
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# Convert y_test to int for comparison
y_test = y_test.astype(int)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


In [ ]:
#MultilingualBert Korean
!pip install tensorflow_text
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

uploaded = files.upload()
df = pd.read_excel(pd.ExcelFile(list(uploaded.keys())[0]), header=0)

df.head(5)

X_train, X_test, y_train, y_test = train_test_split(df['tweets'], df['labels'], stratify=df['labels'])

X_train = X_train.astype(str)
X_test = X_test.astype(str)

y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

bert_preprocess = hub.KerasLayer("https://www.kaggle.com/models/keras/bert/frameworks/Keras/variations/bert_base_multi")
bert_encoder = hub.KerasLayer("https://www.kaggle.com/models/keras/bert/frameworks/Keras/variations/bert_base_multi")

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

model = tf.keras.Model(inputs=[text_input], outputs=[l])
model.summary()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5)

# Predict on the testing data
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# Convert y_test to int for comparison
y_test = y_test.astype(int)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


In [ ]:
#MultilingualBert Spanish
!pip install tensorflow_text
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

uploaded = files.upload()
df = pd.read_csv("spanish.csv", encoding='latin1')  # or specify the correct encoding of your CSV file

df.head(5)

X_train, X_test, y_train, y_test = train_test_split(df['traducido'], df['class'], stratify=df['class'])

X_train = X_train.astype(str)
X_test = X_test.astype(str)

y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

bert_preprocess = hub.KerasLayer("https://www.kaggle.com/models/keras/bert/frameworks/Keras/variations/bert_base_multi")
bert_encoder = hub.KerasLayer("https://www.kaggle.com/models/keras/bert/frameworks/Keras/variations/bert_base_multi")

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

model = tf.keras.Model(inputs=[text_input], outputs=[l])
model.summary()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5)

# Predict on the testing data
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# Convert y_test to int for comparison
y_test = y_test.astype(int)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
